#### Importing Required Libraries

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "ggplot2" 

#### Importing and Exploring Data

In [2]:
df = pd.read_csv("supply_chain_data.csv") # Load the file
pd.options.display.max_columns = 60 # To visualize all the columns in the dataframe
df.head(3) # Visualize first three rows

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,Shipping times,Shipping carriers,Shipping costs,Supplier name,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,4,Carrier B,2.956572,Supplier 3,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,2,Carrier A,9.716575,Supplier 3,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,2,Carrier B,8.054479,Supplier 1,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282


In [3]:
print(f"The shape of dataframe is {df.shape}")
print("**************************")
print(df.info())
print("**************************")
print(df[df.duplicated()])

The shape of dataframe is (100, 24)
**************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name     

+ There are 100 rows and 24 columns in the dataset.
+ No duplicates and no null values found

## Product wise Analysis

In [4]:
pie_chart= px.pie(df.groupby("Product type").sum().round(3), values= "Number of products sold", names= df.groupby("Product type").sum().round(3).index, 
                  title= "Product wise Contribution in Total Sales", color= df.groupby("Product type").sum().round(3).index,
                  hover_data= ['Number of products sold'])
pie_chart.update_traces(textposition='inside', textinfo='percent+label')
pie_chart.show()

+ Skincare products are sold most which is 45% of overall products sold where as skincare products are sold least

In [5]:
fig = px.bar(df.groupby("Product type").sum().round(3), x= df.groupby("Product type").sum().round(3).index, 
             y= df.groupby("Product type").sum().round(3)['Revenue generated'], color= df.groupby("Product type").sum().round(3).index, title= "Product wise Revenue")
fig.show()

+ Skincare products generate the most revenue followed by haircare products

In [6]:
mean_product_df = df.groupby("Product type").mean().round(3)
px.pie(mean_product_df, values= "Defect rates", names= mean_product_df.index, title= "Avg Product wise Defect rates", color= ['r', 'g', 'b'], hole= 0.5,
       color_discrete_sequence=px.colors.qualitative.Dark24)

+ Avg defect rate of haircare products is highest where as cosmetics has the least defect rate

### Revenue vs Price

In [7]:
px.scatter(df, x= "Price", y= "Revenue generated",color='Product type', trendline= "ols", title= "Revenue vs Price")

+ There is a positive relationship between price and revenue generated for skincare products
+ Slightly negative relationship is shown for haircare and cosmetics products

### Correlation Matrix

In [8]:

correlation_matrix = df[['Price', 'Revenue generated', 'Shipping costs', 'Manufacturing costs', 'Costs']].corr()

fig = px.imshow(correlation_matrix,
                color_continuous_scale="YlGnBu",  # Choose the color scale (change to your preference)
                title="Correlation Heatmap")

# Add custom text annotations to the heatmap
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        fig.add_annotation(
            x=j, y=i,
            text=f"{correlation_matrix.iloc[i, j]:.2f}", showarrow= False
        )

fig.show()


## Analyzing Lead time and Manufacturing Costs
+ Lead time refers to the amount of time it takes for an order to be fulfilled from the moment it is placed until it is delivered to the customer. It encompasses all the processes involved in sourcing, production, transportation, and delivery. Lead time is a critical factor in supply chain management as it affects inventory levels, production planning, and customer satisfaction.
+ Manufacturing costs include all the expenses associated with producing a product. These costs can be categorized into three main types (Direct Material costs, Direct Labour costs, Overhead costs)

In [9]:
avg_leadtime_manucosts = df.groupby("Product type").mean()[['Lead time', 'Manufacturing costs']].round(3).reset_index().rename(
    columns= {'Lead time': "Avg Lead time", 'Manufacturing costs': "Avg Manufacturing costs"}
)

melted_df = pd.melt(avg_leadtime_manucosts, id_vars=['Product type'], value_vars=['Avg Lead time', 'Avg Manufacturing costs'],
                    var_name='Metric', value_name='Value')

fig = px.bar(melted_df, x='Product type', y='Value', color='Metric', barmode='group',
             text='Value', title='Average Lead Time and Average Manufacturing Costs by Product Type')

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.update_traces(textposition='outside')

fig.show()


+ Avg Lead time of haircare products is highest among all product types
+ Avg Manufacturing cost of skincare products is highest among all product types

## SKU (STOCK KEEPING UNITS) Analysis
+ SKU stands for "Stock Keeping Unit." In supply chain management and retail operations, an SKU is a unique identifier assigned to each distinct product and item in a company's inventory. It serves as a way to uniquely identify and track individual products within the inventory system.

In [10]:
# Revenue generated by SKU
sku_revenue = px.bar(df, x= "SKU", y= "Revenue generated", color= "SKU", title= "Revenue generated by each SKU")
sku_revenue.update_layout(showlegend= False)
sku_revenue.show()

+ SKU51 generated the most revenue where as SKU59 generated the least revenue

In [11]:
# Stock levels represent the quantity of products or materials a company holds in its inventory at a specific point in time
# Stock Levels by SKU
stock_levels = px.bar(df, x= "SKU", y= "Stock levels", color= "SKU", title= "Revenue generated by each SKU")
stock_levels.update_layout(showlegend= False)
stock_levels.show()

+ SKU12, SKU52 and SKU59 has the most stock availability
+ There is no stock available for SKU68

In [12]:
# Order quantity refers to the quantity of goods or products a company decides to purchase or produce in a single order
sku_orderquantity = px.bar(df, x= "SKU", y= "Order quantities", color_discrete_sequence=px.colors.qualitative.Pastel,
                           title= "Order quantities by SKU")
sku_orderquantity.show()

+ Order quanities of SKU0 and SKU91 is the highest whereas SKU74 has the least order quantity

# Carrier Analysis

In [13]:
carrier_costs = df.groupby("Shipping carriers").mean()['Shipping costs'].round(3).reset_index()
px.bar(carrier_costs, x= "Shipping carriers", y= "Shipping costs", color= "Shipping carriers", title= "Avg Shipping Carrier cost")

In [14]:
carrier_costs = df.groupby("Shipping carriers").sum()['Shipping costs'].round(3).reset_index()
px.bar(carrier_costs, x= "Shipping carriers", y= "Shipping costs", color= "Shipping carriers", title= "Total Shipping Carrier cost")

+ It can be seen that carrier B has the high shipping costs followed by carrier C but there is not much difference in avg shipping costs of all the carriers

## Transportatin mode analysis

In [15]:
# Cost Distribution by Transportation Mode

transportation_chart = px.pie(df, values='Costs', names='Transportation modes', title='Cost Distribution by Transportation Mode',
                              color_discrete_sequence=px.colors.qualitative.Pastel)
transportation_chart.update_traces(textinfo= "percent+label")
transportation_chart.update_layout(showlegend= False)
transportation_chart.show()

+ Most of the products are transported using road and least by Sea

In [16]:
# Avg Defect rates by Transportation Mode

avg_defect_transport = df.groupby("Transportation modes").mean()['Defect rates'].reset_index()
avg_defect_transport_chart = px.pie(avg_defect_transport, names= "Transportation modes", values= "Defect rates",
                                    color_discrete_sequence=px.colors.qualitative.Set1, hole= 0.4,
                                    title= "Avg Defect rates by Transportation Modes")
avg_defect_transport_chart.show()

+ Avg Defect rates by road is the highest compared to other modes where as least defect rate is of Air mode

## Location Analysis

In [17]:
location_data = df.groupby(["Location", "Product type"]).sum().reset_index()
px.bar(location_data, x= "Location", y= "Revenue generated", barmode= "group", color= "Product type")

+ Revenue generated by skincare products is most in Kolkata followed by Chennai
+ Hair care products produces most revenue from Bangalore and least from Delhi
+ Skincare products generate highest revenue from Mumbai followed by Delhi